# Assignment 6

In [2]:
import sympy as sym

from sympy.physics.mechanics import *
from IPython.display import Math, display

init_vprinting()

## Weather in the land of Oz 

> Oz nevers have two nice days in a row

> If it is a nice day, there will be a 50% chance of rain or 50% chance of snow the next day

> If it is a snow or rain day, there is a 50% chance of having the same, and 25% for the other weather

We can model this using a stochastic matrix (P):

$$P = \frac{1}{4} \begin{bmatrix}
    0 & 2 & 2 \\
    1 & 2 & 1 \\
    1 & 1 & 2 
\end{bmatrix}^{}$$


In [20]:
C = sym.Rational(1, 4) # Constant factored out from our stochastic matrix
P = sym.Matrix([[0, 2, 2], [1, 2, 1], [1, 1, 2]]) # Matrix of possibilities (our stochastic matrix)
x1 = sym.Matrix([1, 0, 0]).T # Vector for good day
x2 = sym.Matrix([0, 1, 0]).T # Vector for rainy day
x3 = sym.Matrix([0, 0, 1]).T # Vector for snow day

def round_expr(expr, num_digits):
    return expr.xreplace({n : round(n, num_digits) for n in expr.atoms(sym.Number)})

# Returns the powers of the stochastic matrix
def Prob(n, A=C, B=P):
    a = C ** n
    b = B ** n 
    c = a*b
    d1 = Math(r'P^{0} ='.format({n}) + sym.latex(a) + sym.latex(b))
    d2 = Math(r'P^{0} ='.format({n}) + sym.latex(c))
    d3 = Math(r'P^{0} ='.format({n}) + sym.latex(round_expr(c, 4)))
    return d1, d2, d3

# Returns the possibilities of weather on a given day 
def Loc_prob(n, X, A=C, B=P):
    a = C ** n
    b = X * B ** n 
    c = a*b
    d1 = Math(r'x_{0} ='.format({n}) + sym.latex(a) + sym.latex(b))
    d2 = Math(r'x_{0} ='.format({n}) + sym.latex(c))
    d3 = Math(r'x_{0} ='.format({n}) + sym.latex(round_expr(c, 4)))
    return d1, d2, d3

### a) Weather two days after a good day

To find the possibility of a nice day two days after a nice day, we will multiply a vector representing a nice day ($\vec{x}$) by $P^2$: 

$$\vec{x}(t=2) = \vec{x}(t=0) P^2$$

$$\vec{x}(t=0) = [1 \ \  0 \ \ 0]$$

In [19]:
Loc_prob(2, x1, C, P)[1]

<IPython.core.display.Math object>

We can see that if the first day was Monday, and it happened to be a nice day, the chances of a nice day on Wendsday is $25\%$.

### b) Fraction of days that rain in Oz

To evaluate the probability distribution of the weather, hence the fraction of the days of a certain weather occuring, we take the limit of $P^n$ as $n \rightarrow \infty$:

$$\lim_{x \rightarrow \infty} P^n = A D^n A^{-1}$$

Where: 

> $A$ is the matrix consisting of the eigenvectors of $P$

> $D$ is a diagonlized matrix of $A$ ($D = A^{-1} P A$)

In [28]:
P0 = C*P
A, D = P0.diagonalize() # Find eigenvector matrix and diagonalize P
display(Math(r'A =' + sym.latex(A)))
display(Math(r'D =' + sym.latex(D)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

We can see that:
$$\lim_{n \rightarrow \infty} D^n =
\begin{bmatrix}
    0 & 0 & 0 \\
    0 & 0 & 0 \\
    0 & 0 & 1
\end{bmatrix}$$

$$\therefore \lim_{n \rightarrow \infty} P^n = A^{-1}\begin{bmatrix}
    0 & 0 & 0 \\
    0 & 0 & 0 \\
    0 & 0 & 1
\end{bmatrix} A$$

In [38]:
Dn = sym.Matrix([[0, 0, 0], [0, 0, 0], [0, 0, 1]])
Pn = A * Dn * A**(-1)
display(Math(r'\lim_{n \rightarrow \infty} P^n =' + sym.latex(Pn)))

<IPython.core.display.Math object>

As we can see, Oz rains $\frac{2}{5}$ of the days.

\newpage
## Ising model in one dimension 

$$E = - J \sum_{n=1}^N \sigma_n \sigma_{n+1}$$

Where $\sigma$ can be either $+1$ or $-1$.

### a) The partition function $Z$

$$Z = \sum_\mu e^{-\beta E_\mu} = \sum_{\sigma_i} e^{\beta J \sum_{n=1}^N \sigma_n \sigma_{n+1}}$$

Let's define a transfer matrix $T$: 

$$T = \begin{bmatrix}
    e^{\beta J} & e^{-\beta J} \\
    e^{-\beta J} & e^{\beta J}
\end{bmatrix}$$

> Where the indicies correspond to $\sigma_n$ and $\sigma_{n+1}$

The partition fuction then becomes: 

$$\begin{align}
Z &= \sum_{\sigma_i} \prod_{n=1}^N \vec{\sigma}_n T \vec{\sigma}_{n+1} \\
    &= \sum_{\sigma_i} \vec{\sigma}_1 T \vec{\sigma}_{n+1} \\
    &= \sum_{\sigma_1} \vec{\sigma}_1 T \vec{\sigma}_{1} \\ 
    &= \text{Tr}(T^N) \\
    &= \text{Tr}(A D^N A)\\
    &= \text{Tr}(D^N) \\
    &= \sum_i \lambda_i^N
    \end{align}$$

Where: 

> $D$ is the diagonalized matrix of $T$

> $A$ is the matrix containing the eigenvertors of $T$

> $\lambda_i$ are the eigen values of T

In [40]:
# Define the matrix T
bet, J = sym.symbols('beta J')
T = sym.Matrix([[sym.exp(bet * J), sym.exp(-bet*J)], [sym.exp(-bet*J), sym.exp(bet * J)]])

In [45]:
# Find the eigenvalues and eigenvectors 
T_evec, T_eval = T.diagonalize()
display(Math(r'D =' + sym.latex(T_eval)))

<IPython.core.display.Math object>

Now that we have the eigenvalues, we can see that the partition function: 

$$\begin{align}
Z &= \left(-\sqrt{e^{-2J\beta}} + e^{J\beta} \right)^N + \left(\sqrt{e^{-2J\beta}} + e^{J\beta} \right)^N \\
  &= \left(-e^{-J\beta} + e^{J\beta} \right)^N + \left( e^{-J\beta} + e^{J\beta} \right)^N \\
  &= 2 \cosh^N(J\beta) + 2 \sinh^N(J\beta)
  \end{align}$$

### b) Graph of the heat capacity ($C_V$) as a function of temperature ($T$) for large $N$